In [2]:
import os 
import sys
import pickle
import networkx as nx
import pandas as pd
import numpy as np

import itertools
import sklearn
import torch
import datetime
import matplotlib, matplotlib.pyplot as plt
from collections import defaultdict

from torch.autograd import Variable
from models.model_wrapper import MLP
from data import datasets
from data.gene_graphs import GeneManiaGraph, RegNetGraph
from data.utils import record_result
from data.clinical.datasets import TCGADataset, Task
from data.clinical import taskloader
from data.clinical import split_dataset

from models.model_wrapper import MLP, GCN, SLR

In [4]:
results

,task,auc,gene,model,graph,seed,train_size,optimize_graph_results
0,_EVENT-BRCA,0.503511,NaN,GCN_lay20_chan32_emb32_dropout_pool_kmeans,genemania,0,50,None
1,_EVENT-BRCA,0.527005,NaN,GCN_lay20_chan32_emb32_dropout_pool_kmeans,genemania,1,50,None
2,_EVENT-BRCA,0.507555,NaN,GCN_lay20_chan32_emb32_dropout_pool_kmeans,genemania,2,50,None


In [2]:
tcga = TCGADataset()
task_id = "_EVENT-BRCA"
tasks = [Task(tcga, task_id, limit=1000)]

Torrent name: TCGA_tissue_ppi.hdf5, Size: 1748.32MB


In [3]:
graphs = {"genemania": GeneManiaGraph()}


Torrent name: genemania.pkl, Size: 9.61MB
Checking for pieces on disk: |██████████████████████████████████████████████████| 100.0% 
Found 294 finished pieces out of 294 total pieces.
Found dataset at /home/martin_clyde_weiss/.academictorrents-datastore/genemania.pkl


In [3]:
# Setup the results dictionary
filename = "experiments/results/graph-gen-clinical.pkl"
try:
    results = pickle.load(open(filename, "rb"), encoding='latin1')
    print("Loaded Checkpointed Results")
except Exception as e:
    print(e)
    results = pd.DataFrame(columns=['task', 'auc', 'gene', 'model', 'graph', 'seed', 'train_size', 'optimize_graph_results'])
    print("Created a New Results Dictionary")


Loaded Checkpointed Results


In [5]:
train_size = 50
test_size = 500
trials = 3
cuda = False
models = [           
              GCN(name="GCN_lay20_chan32_emb32_dropout_pool_kmeans", cuda=cuda, num_layer=4, channels=32, embedding=32, prepool_extralayers=5, pooling="kmeans"),
              GCN(name="GCN_lay20_chan32_emb32_dropout_pool_hierarchy", cuda=cuda, num_layer=4, channels=32, embedding=32, prepool_extralayers=5, pooling="hierarchy"),
              GCN(name="GCN_lay20_chan32_emb32_dropout_pool_random", cuda=cuda, num_layer=4, channels=32, embedding=32, prepool_extralayers=5, pooling="random"),
              MLP(name="MLP_lay2_chan512_dropout", cuda=cuda, dropout=True, num_layer=2, channels=512),
              MLP(name="MLP_lay2_chan512", cuda=cuda, dropout=False, num_layer=2, channels=512),
              SLR(name="SLR_lambda1_l11", cuda=cuda)
             ]

In [6]:
# Create the set of all experiment ids and see which are left to do
columns = ["task", "graph", "model", "seed", "train_size"]
all_exp_ids = [x for x in itertools.product(tasks, graphs.keys(), models, range(trials), [train_size])]
all_exp_ids = pd.DataFrame(all_exp_ids, columns=columns)
all_exp_ids.index = ["-".join(map(str, tup[1:])) for tup in all_exp_ids.itertuples(name=None)]
results_exp_ids = results[columns].copy()
results_exp_ids.index = ["-".join(map(str, tup[1:])) for tup in results_exp_ids.itertuples(name=None)]
intersection_ids = all_exp_ids.index.intersection(results_exp_ids.index)
todo = all_exp_ids.drop(intersection_ids).to_dict(orient="records")

print("todo: " + str(len(todo)))
print("done: " + str(len(results)))

todo: 18
done: 0


In [7]:
def optimize_graph(model, gene_graph, gene, X_train, X_test, y_train, y_test):
    neighbors = list(gene_graph.first_degree(gene)[0])
    neighbors = [n for n in neighbors if n in X_train.columns.values]
    res = {}
    for neighbor in neighbors:
        candidate_nodes = neighbors if gene == neighbor else set(neighbors) - set(neighbor)
        x_train = X_train.loc[:, candidate_nodes].copy()
        x_test = X_test.loc[:, candidate_nodes].copy()
        x_train[gene] = 1
        x_test[gene] = 1

        try:
            model.fit(x_train, y_train)
            x_test = Variable(torch.FloatTensor(np.expand_dims(x_test.values, axis=2)), requires_grad=False).float()
            if cuda:
                x_test = x_test.cuda()
            y_hat = model.predict(x_test)[:, 1].data.cpu().numpy()
            auc = sklearn.metrics.roc_auc_score(y_test, np.asarray(y_hat).flatten())
            res[neighbor] = auc
            model.best_model = None # cleanup

        except Exception as e:
            print(e)
    return res


In [ ]:
optimize_graph_results = []

for row in todo:
    if len(results) % 10 == 0:
        print(len(results))
    task = row["task"]
    graph_name = row["graph"]
    seed = row["seed"]
    model = row["model"]

    experiment = {
        "task": task.id,
        "model": model.name,
        "graph": graph_name,
        "seed": seed,
        "train_size": train_size,
        "optimize_graph_results": None,
    }

    try:
        X_train, X_test, y_train, y_test = sklearn.model_selection.\
            train_test_split(task.data, task.labels, stratify=task.labels, 
                             train_size=train_size, test_size=test_size)
    except ValueError:
        import pdb; pdb.set_trace()
        results = record_result(results, experiment, filename)
        continue

    X_train = X_train.copy()
    X_test = X_test.copy()

    try:
        gene_graph = graphs[graph_name]
        adj = np.asarray(nx.to_numpy_matrix(gene_graph.nx_graph))
        model.fit(X_train, y_train.astype("uint8"), adj=adj)

        x_test = Variable(torch.FloatTensor(np.expand_dims(X_test.values, axis=2)), requires_grad=False).float()
        if cuda:
            x_test = x_test.cuda()
        y_hat = model.predict(x_test)[:, 1].data.cpu().numpy()
        auc = sklearn.metrics.roc_auc_score(y_test.astype("uint8"), np.asarray(y_hat).flatten())
        model.best_model = None # cleanup
        experiment["auc"] = auc
    except Exception as e:
        import pdb; pdb.set_trace()
        print(e)
        
    results = record_result(results, experiment, filename)


0


In [19]:
from sklearn.cluster import KMeans
adj = np.asarray(nx.to_numpy_matrix((graphs["genemania"].nx_graph)))
ids = KMeans(n_clusters=1000, init='k-means++', n_init=10, max_iter=300, tol=0.0001, precompute_distances='auto', verbose=0, random_state=None, copy_x=True, n_jobs=-1, algorithm='auto').fit(adj[:2000, :2000])

In [ ]:
ids = KMeans(n_clusters=1000, init='k-means++', n_init=10, max_iter=300, tol=0.0001, precompute_distances='auto', verbose=0, random_state=None, copy_x=True, n_jobs=-1, algorithm='auto').fit(adj[:2000, :2000])

In [21]:
ids = sklearn.cluster.AgglomerativeClustering(n_clusters=50, affinity='euclidean',
                                                     memory='.cache', connectivity=(adj[:100, :100] > 0.).astype(int),
                                                     compute_full_tree='auto', linkage='ward').fit_predict(adj[:100, :100])


/home/martin_clyde_weiss/gene-graph-conv/venv/lib/python3.5/site-packages/sklearn/cluster/hierarchical.py:244: UserWarning: the number of connected components of the connectivity matrix is 94 > 1. Completing it to avoid stopping the tree early.
  affinity='euclidean')


In [25]:
ids

array([ 6,  6,  6,  6,  1,  1,  1, 31,  1,  2, 39,  2,  2,  2,  0,  0,  0,
        0, 49, 49, 29, 29, 48, 48, 30, 30, 23, 23, 45, 45, 46, 46, 14, 14,
       22, 22, 28, 28, 35, 44, 44, 13, 13, 21, 21, 10, 10, 42, 42, 24, 24,
       41, 41, 32, 32, 40, 40, 26, 26, 19, 19, 15, 15, 38, 38, 12, 12, 37,
       27, 33, 37, 11, 11, 36, 47, 25, 36, 20, 20, 17, 17,  9,  9, 34, 34,
       18, 18, 16, 16,  4, 43,  4,  7,  7,  8,  8,  3,  3,  5,  5])